In [1]:
from gensim.models import FastText

In [3]:
import pandas as pd

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
from pathlib import Path

In [5]:
data_path = Path('/data')

In [6]:
full_notes = pd.read_parquet("/data/full_note_text_2017-01-01_2019-06-26.parquet")

In [24]:
full_notes.columns

Index(['provider', 'ProgressNoteID', 'PatientID', 'FacilityID',
       'ProgressNoteType', 'CreatedDate', 'SectionSequence', 'Section',
       'NoteText'],
      dtype='object')

In [12]:
env = 'dev'

In [13]:
import boto3
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [17]:
import json

In [31]:
session = boto3.session.Session()
client = session.client(service_name="secretsmanager", region_name='us-east-1')

avante_db_info = json.loads(
    client.get_secret_value(SecretId="avantedb")["SecretString"]
            )

avante_url = URL(
    drivername="mssql+pyodbc",
    username=avante_db_info["username"],
    password=avante_db_info["password"],
    host=avante_db_info["host"],
    port=avante_db_info["port"],
    database=avante_db_info["dbname"],
    query={"driver": "ODBC Driver 17 for SQL Server"},
            )

In [21]:
note_types = set('eMAR-Medication Administration Note	* General NURSING Note	* Skilled Nursing Note	Weekly Nurses Skin Observation Note	Physician Progress Notes	Braden Data Tool	X Social Service Interview	Dietary RD/DTR Data Collection Progress Note	z R.T. Shift Note (7am-7pm)	z R.T. Shift Note (7pm - 7am)	* Dietary RD/DTR Progress Note	* Social Services Note	* Activity Note	* Physician Progress Note	eMar - Shift Level Administration Note	* Weekly Wound Documentation	* Activities Admission/Readmission NOte	* Weight Meeting Note	BIMS Data Results	* Incident/Accident Note	* Vent/Trach Clinical Observation Note	* Admission Note	MDS Note	* Skin / Wound Note	* Skin / Wound Note	* Skin / Wound Note	Global Deterioration Scale Note'.split('\t'))

In [22]:
note_types

{'* Activities Admission/Readmission NOte',
 '* Activity Note',
 '* Admission Note',
 '* Dietary RD/DTR Progress Note',
 '* General NURSING Note',
 '* Incident/Accident Note',
 '* Physician Progress Note',
 '* Skilled Nursing Note',
 '* Skin / Wound Note',
 '* Social Services Note',
 '* Vent/Trach Clinical Observation Note',
 '* Weekly Wound Documentation',
 '* Weight Meeting Note',
 'BIMS Data Results',
 'Braden Data Tool',
 'Dietary RD/DTR Data Collection Progress Note',
 'Global Deterioration Scale Note',
 'MDS Note',
 'Physician Progress Notes',
 'Weekly Nurses Skin Observation Note',
 'X Social Service Interview',
 'eMAR-Medication Administration Note',
 'eMar - Shift Level Administration Note',
 'z R.T. Shift Note (7am-7pm)',
 'z R.T. Shift Note (7pm - 7am)'}

In [45]:
a_engine=create_engine(avante_url)

In [50]:
pd.read_sql("SELECT TOP 10 CreatedDate, ProgressNoteID FROM dbo.view_ods_progress_note WHERE YEAR(CreatedDate) = 2012", a_engine)

,CreatedDate,ProgressNoteID
0,2012-10-08 16:22:11.993,282170
1,2012-11-01 09:42:09.797,383360
2,2012-11-01 09:42:09.797,383360
3,2012-11-01 09:44:39.867,383397
4,2012-11-01 09:43:13.693,383472
5,2012-11-01 09:43:13.693,383472
6,2012-11-01 09:43:13.693,383472
7,2012-11-01 09:43:13.693,383472
8,2012-11-01 09:43:13.693,383472
9,2012-11-01 09:43:13.693,383472


In [56]:
years = range(2011, 2020)

In [81]:
f"""SELECT TOP 10 'avante' as client, ProgressNoteID, PatientID, FacilityID,
       ProgressNoteType, CreatedDate, SectionSequence, Section, NoteText
       FROM view_ods_progress_note
       WHERE YEAR(CreatedDate) = {2017}
       AND ProgressNoteType IN ({", ".join([f"'{n}'" for n in note_types])})
       """

"SELECT TOP 10 'avante' as client, ProgressNoteID, PatientID, FacilityID,\n       ProgressNoteType, CreatedDate, SectionSequence, Section, NoteText\n       FROM view_ods_progress_note\n       WHERE YEAR(CreatedDate) = 2017\n       AND ProgressNoteType IN ('* Skin / Wound Note', '* Social Services Note', 'eMAR-Medication Administration Note', 'Global Deterioration Scale Note', '* Weight Meeting Note', 'BIMS Data Results', '* Incident/Accident Note', '* Activities Admission/Readmission NOte', 'MDS Note', 'Physician Progress Notes', 'eMar - Shift Level Administration Note', '* Weekly Wound Documentation', 'Braden Data Tool', '* Physician Progress Note', '* Skilled Nursing Note', 'Weekly Nurses Skin Observation Note', 'z R.T. Shift Note (7pm - 7am)', '* Dietary RD/DTR Progress Note', '* Vent/Trach Clinical Observation Note', 'z R.T. Shift Note (7am-7pm)', '* General NURSING Note', '* Activity Note', 'Dietary RD/DTR Data Collection Progress Note', '* Admission Note', 'X Social Service Inter

In [87]:
def connect_and_pull_notes(year):
    engine = create_engine(avante_url)
    query = f"""SELECT 'avante' as client, ProgressNoteID, PatientID, FacilityID,
       ProgressNoteType, CreatedDate, SectionSequence, Section, NoteText
       FROM view_ods_progress_note
       WHERE YEAR(CreatedDate) = {year}
       AND ProgressNoteType IN ({", ".join([f"'{n}'" for n in note_types])})
       """
       
    print(year, '\n\n')
    print(query)
    
    notes = pd.read_sql(query, engine)
    notes.to_parquet(f'/data/avante_notes_{year}.parquet')
    return notes
    

In [88]:
pool = ThreadPool(3)

In [ ]:
note_dfs = pool.map(connect_and_pull_notes, years)

2011 2012 


SELECT 'avante' as client, ProgressNoteID, PatientID, FacilityID,
       ProgressNoteType, CreatedDate, SectionSequence, Section, NoteText
       FROM view_ods_progress_note
       WHERE YEAR(CreatedDate) = 2012
       AND ProgressNoteType IN ('* Skin / Wound Note', '* Social Services Note', 'eMAR-Medication Administration Note', 'Global Deterioration Scale Note', '* Weight Meeting Note', 'BIMS Data Results', '* Incident/Accident Note', '* Activities Admission/Readmission NOte', 'MDS Note', 'Physician Progress Notes', 'eMar - Shift Level Administration Note', '* Weekly Wound Documentation', 'Braden Data Tool', '* Physician Progress Note', '* Skilled Nursing Note', 'Weekly Nurses Skin Observation Note', 'z R.T. Shift Note (7pm - 7am)', '* Dietary RD/DTR Progress Note', '* Vent/Trach Clinical Observation Note', 'z R.T. Shift Note (7am-7pm)', '* General NURSING Note', '* Activity Note', 'Dietary RD/DTR Data Collection Progress Note', '* Admission Note', 'X Social Service Inte

In [ ]:
avante_notes = pd.concat(note_dfs).reset_index(drop=True)

In [91]:
avante_notes.sample(10)

,client,ProgressNoteID,PatientID,FacilityID,ProgressNoteType,CreatedDate,SectionSequence,Section,NoteText
4983691,avante,6320054,251673,20,eMAR-Medication Administration Note,2014-07-21 05:18:40.853,1,Note Text,Norco Tablet 5-325 MG\n\tGive 1 tablet by mou...
14265321,avante,19821886,463552,15,eMAR-Medication Administration Note,2017-08-05 20:05:49.243,1,Note Text,OxyCODONE HCl Tablet 5 MG\n\tGive 2 tablet by...
3899,avante,59411,247,4,* General NURSING Note,2012-06-21 14:34:24.300,1,Note Text,continent of bowel and bladder. Peri care rend...
11807930,avante,16194204,397674,7,eMAR-Medication Administration Note,2016-10-03 08:08:49.633,1,Note Text,Enoxaparin Sodium Solution 40 MG/0.4ML\n\tInj...
10822644,avante,14711802,374022,14,eMAR-Medication Administration Note,2016-05-25 22:22:04.653,1,Note Text,\n\nPRN Administration was: Effective\n
16261231,avante,22922332,514807,20,* General NURSING Note,2018-04-24 03:06:07.647,1,Note Text,afety measures in place. Call bell and fluids ...
4371910,avante,5497086,229637,8,* General NURSING Note,2014-05-11 22:16:05.497,1,Note Text,pt is alert with confusion. no complaints of p...
13010461,avante,17957819,372951,8,Dietary RD/DTR Data Collection Progress Note,2017-03-03 10:31:34.297,1,Note Text,darone HCL. Lab data collected from Medical hi...
10744242,avante,14592918,374508,3,* Skilled Nursing Note,2016-05-15 11:06:15.023,1,Note Text,"s, pt educated and has been easily re-directed..."
17830083,avante,25271232,629677,9,* Admission Note,2019-03-06 16:01:11.860,1,Note Text,Members of the IDT met with Mr. Hinthorne to i...


In [ ]:
del